In [98]:
import numpy as np
from pprint import pprint

dtype = [
    ('Timestep', float),
    ('Id', int),
    ('Role_name', 'U8'),  
    ('Lane_id', int),  
    ('Location_x', float),
    ('Location_y', float),
    ('Location_z', float),
    ('Rotation_pitch', float),
    ('Rotation_yaw', float),
    ('Rotation_roll', float),
    ('Angular_velocity_x', float),
    ('Angular_velocity_y', float),
    ('Angular_velocity_z', float),
    ('Velocity_x', float),
    ('Velocity_y', float),
    ('Velocity_z', float),
    ('Acceleration_x', float),
    ('Acceleration_y', float),
    ('Acceleration_z', float),
    ('Distance2Ego', float)
]

data = np.genfromtxt('/home/weidong/Documents/data_2024_22_04_13_24_40.csv', dtype=dtype, delimiter=',', skip_header=1)
data['Location_y'] = - data['Location_y']
data['Timestep'] -= data['Timestep'][0] 
data['Timestep'] = np.round(data['Timestep'],2)
ego_rows = data[data['Role_name'] == 'ego']
target_rows = data[data['Role_name'] != 'ego']

In [99]:
# Assert if Collision
assert not np.any((target_rows['Distance2Ego']<=0))

In [100]:
# Assert if exceed Acceleration
acceleration_x = ego_rows['Acceleration_x']
acceleration_y = ego_rows['Acceleration_y']
acceleration_z = ego_rows['Acceleration_z']

acc = np.sqrt(acceleration_x**2 + acceleration_y**2)
assert np.max(acc) > 10

In [ ]:
# Assert if distance < Value
assert np.any((target_rows['Distance2Ego']< 1.5))

In [ ]:
# Assert if AD overtake 
# x > 0 in front, y > 0 at left
# assume same heading, same lane

ego_c = ego_rows[['Location_x','Location_y','Rotation_yaw',]]
target_c = target_rows[['Location_x','Location_y','Rotation_yaw']]


import numpy as np
import math

def rotate_point(point, angle):
    """Rotate a point around the origin (0, 0) by a given angle in radians."""
    x, y = point
    cos_angle = math.cos(angle)
    sin_angle = math.sin(angle)
    rotated_x = x * cos_angle - y * sin_angle
    rotated_y = x * sin_angle + y * cos_angle
    return rotated_x, rotated_y

def transform_points(p1_array, p2_array):
    """Transform points in p2_array from world coordinates to local coordinates of p1_array."""
    transformed_points = []
    for p1, p2 in zip(p1_array, p2_array):
        # Translate p2 to the origin of p1
        translated_x = p2[0] - p1[0]
        translated_y = p2[1] - p1[1]

        # Convert yaw angle from degrees to radians
        yaw_radians = math.radians(p1[2])

        # Rotate p2 around the origin of p1 by the yaw angle of p1
        rotated_x, rotated_y = rotate_point((translated_x, translated_y), yaw_radians)

        transformed_points.append((rotated_x, rotated_y))

    return np.array(transformed_points)

# Transform p2_array to the local coordinate system of p1_array
transformed_points = transform_points(ego_c, target_c)

assert np.all(transformed_points[:,0]>0)

In [126]:
# Assert safe distance
# D >= v_ad * t_front
# v_ad:t_front (0-2):1.0 | (2-2.78):1.1 | (2.78-5.56):1.2 | (5.56-8.33):1.3 | (8.33-11.11):1.4 | (11.11-13.89):1.5 | (13.89-16.67):1.6 
# Assume same heading, same lane
ego_v = np.sqrt(ego_rows['Velocity_x']**2 + ego_rows['Velocity_y']**2 + ego_rows['Velocity_z']**2)
# target_v = target_rows[['Velocity_x','Velocity_y','Velocity_z','Distance2Ego']]
distance_2_ego = target_rows['Distance2Ego']


d_min = []
for v in ego_v:
    if 0 <= v < 2:
        d_min.append(v*1.0)
    elif 2 <= v < 2.78:
        d_min.append(v*1.1)
    elif 2.78<= v < 5.56 :
        d_min.append(v*1.2)
    elif 5.56 <= v < 8.33:
        d_min.append(v*1.3)
    elif 8.33 <= v < 11.11:
        d_min.append(v*1.4)
    elif 11.11 <= v < 13.89:
        d_min.append(v*1.5)
    elif 13.89 <= v < 16.67:
        d_min.append(v*1.6)

d_min = np.array(d_min)
assert np.all(distance_2_ego>=d_min)

(array([209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,
        222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234,
        235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247,
        248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260,
        261]),)